In [ ]:
! git clone https://github.com/SAI990323/TALLRec.git

Cloning into 'TALLRec'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 219 (delta 69), reused 79 (delta 64), pack-reused 128 (from 1)
Receiving objects: 100% (219/219), 5.77 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [ ]:
!pip install peft==0.3.0 fire datasets bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.9.0
    Uninstalling peft-0.9.0:
      Successfully uninstalled peft-0.9.0


In [ ]:
! chmod 777 ./TALLRec/shell/instruct_7B.sh

In [ ]:
%cd TALLRec

/content/TALLRec


In [ ]:
! ./shell/instruct_7B.sh 0 42

0, 42
lr: 1e-4, dropout: 0.05 , seed: 42, sample: 64
2024-11-14 11:38:56.743624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 11:38:56.777564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 11:38:56.787481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 11:38:56.810469: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import sys
import fire
import gradio as gr
import torch
torch.set_num_threads(1)
import transformers
import json
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.metrics import roc_auc_score
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass

base_model = 'baffo32/decapoda-research-llama-7B-hf'
model_type = 'alpaca-lora-7b'
load_8bit = False
lora_weights = './alpaca-lora-7B'

tokenizer = LlamaTokenizer.from_pretrained(base_model)
if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        torch_dtype=torch.float16,
        device_map={'': 0}
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        base_model, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        device_map={"": device},
    )

tokenizer.padding_side = "left"
# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

if not load_8bit:
    model.half()  # seems to fix bugs for some users.

model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

########inference#########

instructions=["Given the user's preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\"."]
inputs=["User Preference: \"Paris, Texas (1984)\", \"Rebel Without a Cause (1955)\", \"Return of the Pink Panther, The (1974)\", \"Ace Ventura: Pet Detective (1994)\", \"Magnificent Seven, The (1954)\", \"Star Trek: The Wrath of Khan (1982)\", \"Cat People (1982)\", \"Orlando (1993)\", \"Dave (1993)\"\nUser Unpreference: \"Kalifornia (1993)\"\nWhether the user will like the target movie \"Perez Family, The (1995)\"?"]
temperature=1
top_p=1.0
top_k=40
num_beams=1
max_new_tokens=128
batch_size=1

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""

prompt = [generate_prompt(instruction, input) for instruction, input in zip(instructions, inputs)]
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
generation_config = GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    num_beams=num_beams
)
with torch.no_grad():
    generation_output = model.generate(
        **inputs,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        # batch_size=batch_size,
    )
s = generation_output.sequences
scores = generation_output.scores[0].softmax(dim=-1)
logits = torch.tensor(scores[:,[8241, 3782]], dtype=torch.float32).softmax(dim=-1)
input_ids = inputs["input_ids"].to(device)
L = input_ids.shape[1]
s = generation_output.sequences
output = tokenizer.batch_decode(s, skip_special_tokens=True)
output = [_.split('Response:\n')[-1] for _ in output]

print(output[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you lo

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:372: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(
Asking to truncate to max_length

Yes.

### Output:
Yes.

### Explanation:
The user's preference and unpreference are "Paris, Texas (1984)", "Rebel Without a Cause (1955)", "Return of the Pink Panther, The (1974)", "Ace Ventura: Pet Detective (1994)", "Magnificent Seven, The (1954)", "Star Trek: The Wrath of Khan (1982)", "Cat People (1982)", "


<ipython-input-2-f6721614afcc>:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits = torch.tensor(scores[:,[8241, 3782]], dtype=torch.float32).softmax(dim=-1)
